# Data Visualization: Week 2, Lecture 2

#### About the dataset
- Data obtained using Google Trends https://trends.google.com/trends/explore?date=all&geo=US&q=sunscreen&hl=en
- Access CSV here:  https://drive.google.com/file/d/1DS9_ENBsMZB7Mjd-WljuT7H2YSZ2PtF0/view?usp=sharing
- Dataset as of November 2023 also in repo as `sunscreen_popularity.txt` in `data` subfolder.

#### Learning Objectives
- Identify order of seasonal ARIMA model
- Fit and evaluate seasonal ARIMA models
- Compare multiple ARIMA models using diagnostic plots and evaluation metrics

## Imports

In [ ]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pmdarima as pm
from pmdarima.arima.utils import ndiffs, nsdiffs
from pmdarima.model_selection import train_test_split

from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_error,
    r2_score,
)

import statsmodels.tsa.api as tsa

#set_config(transform_output="pandas")
plt.rcParams["figure.figsize"] = (12, 4)
sns.set_context("talk", font_scale=0.9)

# set random seed
SEED = 321
np.random.seed(SEED)

## Custom Functions

In [ ]:
def plot_forecast(ts_train, ts_test, forecast_df, n_train_lags=None, 
                  figsize=(10,4), title='Comparing Forecast vs. True Data'):
    
### PLot training data, and forecast (with upper/,lower ci)
    fig, ax = plt.subplots(figsize=figsize)

    # setting the number of train lags to plot if not specified
    if n_train_lags == None:
        n_train_lags = len(ts_train)

    # Plotting Training  and test data
    ts_train.iloc[-n_train_lags:].plot(ax=ax, label="train")
    ts_test.plot(label="test", ax=ax)

    # Plot forecast
    forecast_df["mean"].plot(ax=ax, color="green", label="forecast")

    # Add the shaded confidence interval
    ax.fill_between(
        forecast_df.index,
        forecast_df["mean_ci_lower"],
        forecast_df["mean_ci_upper"],
        color="green",
        alpha=0.3,
        lw=2,
    )

    # set the title and add legend
    ax.set_title(title)
    ax.legend()

    return fig, ax


In [ ]:
# Custom function for Ad Fuller Test
def get_adfuller_results(ts, alpha=.05, label='adfuller', **kwargs): #kwargs for adfuller()
    # Saving each output
    (test_stat, pval, nlags, nobs, crit_vals_d, icbest) = tsa.adfuller(ts, **kwargs)
    
    # Converting output to a dictionary with the interpretation of p
    adfuller_results = {
        "Test Statistic": test_stat,
        "# of Lags Used": nlags,
        "# of Observations": nobs,
        "p-value": round(pval, 6),
        "alpha": alpha,
        "sig/stationary?": pval < alpha,
    }
    
    return pd.DataFrame(adfuller_results, index=[label])

In [ ]:
def regression_metrics_ts(ts_true, ts_pred, label="", verbose=True, output_dict=False,):
    # Get metrics
    mae = mean_absolute_error(ts_true, ts_pred)
    mse = mean_squared_error(ts_true, ts_pred)
    rmse = mean_squared_error(ts_true, ts_pred, squared=False)
    r_squared = r2_score(ts_true, ts_pred)
    mae_perc = mean_absolute_percentage_error(ts_true, ts_pred) * 100

    if verbose == True:
        # Print Result with label
        header = "---" * 20
        print(header, f"Regression Metrics: {label}", header, sep="\n")
        print(f"- MAE = {mae:,.3f}")
        print(f"- MSE = {mse:,.3f}")
        print(f"- RMSE = {rmse:,.3f}")
        print(f"- R^2 = {r_squared:,.3f}")
        print(f"- MAPE = {mae_perc:,.2f}%")

    if output_dict == True:
        metrics = {
            "Label": label,
            "MAE": mae,
            "MSE": mse,
            "RMSE": rmse,
            "R^2": r_squared,
            "MAPE(%)": mae_perc,
        }
        return metrics

In [ ]:
def plot_acf_pacf(ts, nlags=40, figsize=(10, 5), 
                  annotate_sig=False, alpha=.05,
                  acf_kws={}, pacf_kws={},  
                  annotate_seas=False, m = None,
                  seas_color='black'):

    fig, axes = plt.subplots(nrows=2, figsize=figsize)

    # Sig lags line style
    sig_vline_kwargs = dict(ls=":", lw=1, zorder=0, color="red")

    # ACF
    tsa.graphics.plot_acf(ts, ax=axes[0], lags=nlags, **acf_kws)

    ## Annotating sig acf lags
    if annotate_sig == True:
        sig_acf_lags = get_sig_lags(ts, nlags=nlags, alpha=alpha, type="ACF")
        for lag in sig_acf_lags:
            axes[0].axvline(lag, label="sig", **sig_vline_kwargs)

    # PACF
    tsa.graphics.plot_pacf(ts, ax=axes[1], lags=nlags, **pacf_kws)

    ## Annotating sig pacf lags
    if annotate_sig == True:
        ## ANNOTATING SIG LAGS
        sig_pacf_lags = get_sig_lags(ts, nlags=nlags, alpha=alpha, type="PACF")
        for lag in sig_pacf_lags:
            axes[1].axvline(lag, label="sig", **sig_vline_kwargs)

    ### ANNOTATE SEASONS
    if annotate_seas == True:
        # Ensure m was defined
        if m is None:
            raise Exception("Must define value of m if annotate_seas=True.")

        ## Calculate number of complete seasons to annotate
        n_seasons = nlags // m

        # Seasonal Lines style
        seas_vline_kwargs = dict(ls="--", lw=1, alpha=0.7, color=seas_color, zorder=-1)

        ## for each season, add a line
        for i in range(1, n_seasons + 1):
            axes[0].axvline(m * i, **seas_vline_kwargs, label="season")
            axes[1].axvline(m * i, **seas_vline_kwargs, label="season")

    fig.tight_layout()

    return fig

## Load and Transform Data

- Load in the data, using `skiprows=[0]`
- Rename the column containing the data to "popularity"
- Convert the data column to a datetime column, then set as index
- Convert `popularity` column to float datatype
- Set time series as variable called `ts`
- Set frequency on time series index as `MS`
- Plot time series and check for nulls

In [ ]:
# sunscreen
df_sun = pd.read_csv('data/sunscreen_popularity.txt', skiprows= [0])
df_sun.head()

In [ ]:
df_sun.rename(
    columns = {'Month': 'date', 'sunscreen: (United States)': 'popularity'},
    inplace = True
)
df_sun.head()

In [ ]:
# Make data a DateTime Object


In [ ]:
# Set index as date


In [ ]:
# Change datatype of popularity to float


In [ ]:
# Define the time series as ts


In [ ]:
# Set the frequency to the start of the month (does not require resampling)


In [ ]:
# Visualize the ts


In [ ]:
# Check for nulls


## Determine Seasonality

- Use `tsa.seasonal_decompose()` to visualize trend and seasonal elements
- Calculate the seasonal delta and account for how much variation it describes
- Plot the seasonal component
- Identify the order of seasonality

In [ ]:
# Apply seasonal decomposition


In [ ]:
# How big is the seasonal component
seasonal_delta = 

# How big is the seasonal component relative to the time series?
print(f"The seasonal component is {seasonal_delta: .2f} which is ~{seasonal_delta/(ts.max()-ts.min()) * 100 :.2f}% of the variation in time series.")

In [ ]:
# Narrow down the date range of the plot


In [ ]:
# From domain knowledge & counting the observations, we identify the order is 12

## Determine Stationarity/Differencing

- Use the augmented Dickey-Fuller to test for stationarity
- If necessary, find the order of differencing using `ndiffs`
- Identify the order of seasonal differencing using `nsdiffs`
- Save a differenced version of the data and plot

In [ ]:
# Testing the raw data for stationarity


In [ ]:
# use ndiffs to determine differencing


In [ ]:
# Determine D


In [ ]:
# apply both differencings


## Determine Initial Model Orders

- Plot the ACF and PACF of the differenced data
- Identify possible model orders

In [ ]:
# We can use our function to highlight the seasonal lags by adding the arguments


In [ ]:
# nonseasonal- PACF drops off after 2
# seasonal - looks like s ACF drops off after 1

## Train Test Split

- Split the data into training and testing subsets. The test set should be one year of data
- Plot the train and test sets

In [ ]:
# tts--goal is to predict next year so 12 months


## Modeling

- Start with an initial model using manually chosen non-seasonal and seasonal orders
- Fit a seasonal ARIMA
- Generate a forecast dataframe
- Use `plot_forecast` and `regression_metrics_ts` to evaluate the model
- Examine `model.summary()`
- Examine `model.plot_diagnostics()`

In [ ]:
# Initial model

# Orders for non seasonal components
p =   # nonseasonal AR
d =   # nonseasonal differencing
q =   # nonseasonal MA

# Orders for seasonal components
P =   # Seasonal AR
D =   # Seasonal differencing
Q =   # Seasonal MA
m =   # Seasonal period

model_1 = 

In [ ]:
# Obtain summary of forecast as dataframe

# Plot the forecast with true values

# Obtain metrics


In [ ]:
# Obtain summary


In [ ]:
# Obtain diagnostic plots


## Fit alternative model using `auto_arima`

- Use `pm.auto_arima` to fit a model using AIC as a selection metric
    - Remember to set `seasonal=True` and `m=12`
    - Use `trace=True` to display metrics
- Check the best model order and seasonal order
- Examine the `model.summary()` output
- Examine `plot_diagnostics()`

In [ ]:
# Default auto_arima will select model based on AIC score


In [ ]:
# the auto_arima will store our best nonseasonal and seasonal orders separtely


In [ ]:
# Obtain summary of the best model from auto_arima


In [ ]:
# Obtain diagnostic plots


## New ARIMA with best orders as by `auto_arima`

- Use `auto_model.order` and `auto_model.seasonal_order` to create an ARIMA model
- Evaluate the model forecast
- Which model is best?

In [ ]:
# Use auto_arima parameters to fit an ARIMA


# Obtain forecast as a dataframe with confidence intervals

# Call the custom function to plot the forecasts with confidence intervals and true values

# Obtain metrics


In [ ]:
# Use auto_arima parameters to fit an ARIMA


# Obtain future forecasts beyond test data


In [ ]:
# Forecast mean:


In [ ]:
# Month with Maximum popularity forecasted for next year


In [ ]:
# Month with Minmum popularity next year


# Final model
The metrics for the initial model and auto model are similar. We choose the auto arima model because it had a slightly lower AIC and BIC, reduced significant correlations in the residuals, and the additional coefficients were significant.